# Part I. ETL Pipeline for Pre-Processing the Files

## CODES FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = '/home/workspace/event_data/*.csv'

file_path_list = []
# Create a for loop to create a list of files and collect each filepath
for files in glob.glob(filepath):
    file_path_list.append(files)

print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
#             print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Data Modeling with Cassandra. 

## Work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# Creating a cluster and establishing a connection to the database

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Creating a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Connect to Keyspace
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Data Model 1 : Produces the following business needs:

#### 1. Extracts the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


In [8]:
## Data model 1:  Extracts the artist, song title and song's length in the music app history \
## by itemInSession and sessionId
query = "CREATE TABLE IF NOT EXISTS music_lib_by_itemin_sessionid "
query = query + "(session_id int, item_in_session int, artist text, song_title text, length float, PRIMARY KEY (item_in_session, session_id))"

try:
    session.execute(query)
except Exception as e:
    print(e)         

In [9]:
## Reads the CSV File and inserts records into the keyspace music_library
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_lib_by_itemin_sessionid (item_in_session, session_id, artist, song_title, length) "
        query = query + "VALUES(%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[3]), int(line[8]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## Validate the  Data Model 1 to verify if the data was entered into the table
query = "SELECT artist, song_title, length FROM music_lib_by_itemin_sessionid WHERE item_in_session = 4 AND session_id = 338"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


## Data Model 2 : Produces the following business needs:

#### 1. Extracts the the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [11]:
## Data model 2:  Extracts the artist, song (sorted by itemInSession) and user (first and last name) in the music app history \
## by userid and sessionid
query = "CREATE TABLE IF NOT EXISTS music_lib_by_userid_sessionid " 
query = query + "(session_id int, user_id int, item_in_session int, artist text, song_title text, first_name text, last_name text, PRIMARY KEY ((user_id, session_id), item_in_session)) WITH CLUSTERING ORDER BY (item_in_session DESC)"

try:
    session.execute(query)
except Exception as e:
    print(e)  

In [12]:
## Reads the CSV File and inserts records into the keyspace music_library
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_lib_by_userid_sessionid (user_id, session_id, item_in_session, artist, song_title, first_name, last_name) "
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[9], line[0], line[1], line[4]))

In [13]:
## Validate the  Data Model 2 to verify if the data was entered into the table
query = "SELECT artist, song_title, first_name, last_name FROM music_lib_by_userid_sessionid WHERE user_id=10 AND session_id=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.first_name, row.last_name)

Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Lonnie Gordon Sylvie Cruz
Kilometer Sebastien Tellier Sylvie Cruz
Greece 2000 Three Drives Sylvie Cruz
Keep On Keepin' On Down To The Bone Sylvie Cruz


## Data Model 3 : Produces the following business needs:

#### 1. Extracts every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [14]:
## Data model 3:  Extracts the user  (first and last name) in the music app history \
## by song title
query = "CREATE TABLE IF NOT EXISTS music_lib_by_song " 
query = query + "(song_title text, session_id int, first_name text, last_name text, PRIMARY KEY (song_title, session_id))"

try:
    session.execute(query)
except Exception as e:
    print(e)  

In [15]:
## Reads the CSV File and inserts records into the keyspace music_library
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_lib_by_song (song_title, session_id, first_name, last_name) "
        query = query + "VALUES(%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[8]), line[1], line[4]))

In [16]:
## Validate the  Data Model 3 to verify if the data was entered into the table
query = "SELECT first_name, last_name FROM music_lib_by_song WHERE song_title='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, row.last_name)

Sara Johnson
Jacqueline Lynch
Tegan Levine


### Drop the tables before closing out the sessions

In [17]:
## TO-DO: Drop the table before closing out the sessions

In [18]:
query = "DROP TABLE music_lib_by_itemin_sessionid"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE music_lib_by_userid_sessionid"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE music_lib_by_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()